In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import transforms

import numpy as np
import matplotlib.pyplot as plt

from scipy.io import loadmat

In [2]:
cuda = 'gpu:0' if torch.cuda.is_available() else 'cpu'
device = torch.device(cuda)
print(torch.cuda.is_available())

False


In [3]:
# Load MAT Data Files

matpath_upper = "/Volumes/RESEARCH1/CAOS/_data/Training/psiTrain/psiTrain1_30km.mat"
matpath_mid = "/Volumes/RESEARCH1/CAOS/_data/Training/psiTrain/psiTrain1_30km_mid.mat"

psi1_upper = loadmat(matpath_upper)
psi1_mid = loadmat(matpath_mid)

psi1_upper_30km_filter = psi1_upper['psi1_30km'].astype(np.float64)
psi1_upper_anomaly = psi1_upper['psi1Anom'].astype(np.float64)
psi1_upper_unfiltered = psi1_upper_30km_filter + psi1_upper_anomaly

psi1_mid_unfiltered = psi1_mid['psi1_mid'].astype(np.float64)
psi1_top_30km = psi1_mid['psi1_top_30km'].astype(np.float64)

In [4]:
class psi1_Dataset:
    """psiTrain 30km mid Dataset."""

    def __init__(self, X, Y, is_train, train_split):
        n_total_samples = len(X[0, 0, :])
        split_idx = int(np.floor(train_split * n_total_samples))

        x_train = X[:, :, :split_idx]
        y_train = Y[:, :, :split_idx]

        x_test = X[:, :, split_idx:]
        y_test = Y[:, :, split_idx:]

        # Differentiate Train v. Test & Normalize
        
        x_mean, x_std = x_train.mean(), x_train.std()
        y_mean, y_std = y_train.mean(), y_train.std()
        
        if is_train:
            self.x = (x_train - x_mean) / x_std
            self.y = (y_train - y_mean) / y_std
        else:
            self.x = (x_test - x_mean) / x_std
            self.y = (y_test - y_mean) / y_std
        
    def __len__(self):
        return len(self.x[0, 0, :])

    def __getitem__(self, idx):
        if torch.is_tensor(idx): idx = idx.tolist()        
        return self.x[:, :, idx], self.y[:, :, idx]

In [5]:
batch_size = 64
train_split = 0.888888

# Create Train/Test datasets
train_dataset = psi1_Dataset(X = psi1_upper_30km_filter,
                             Y = psi1_upper_anomaly,
                             is_train = True,
                             train_split = train_split)

test_dataset = psi1_Dataset(X = psi1_upper_30km_filter,
                            Y = psi1_upper_anomaly,
                            is_train = False,
                            train_split = train_split)

# Create Data Loaders
train_loader = DataLoader(train_dataset,
                          batch_size = batch_size,
                          shuffle = False)

test_loader = DataLoader(test_dataset,
                         batch_size = batch_size,
                         shuffle = False)

In [6]:
class CNN(nn.Module):
    def __init__(self, input_size, output_size):
        
        super(CNN, self).__init__()
        
        self.conv1 = nn.Conv2d( in_channels = 1,   out_channels = 128, kernel_size = 3, bias = False, padding = 1 )
        self.conv2 = nn.Conv2d( in_channels = 128, out_channels = 64,  kernel_size = 3, bias = False, padding = 1 )
        self.conv3 = nn.Conv2d( in_channels = 64,  out_channels = 48,  kernel_size = 3, bias = False, padding = 1 )
        self.conv4 = nn.Conv2d( in_channels = 48,  out_channels = 1,   kernel_size = 3, bias = False, padding = 1 )
        
#         f = np.random.rand(5, 5).astype(np.float32)
#         f = f.reshape(1, 1, f.shape[0], f.shape[1])
#         f = np.repeat(f, 100, axis=1)
#         f = np.repeat(f, 100, axis=0)
#         self.f = nn.Parameter(data=torch.FloatTensor(f), requires_grad=False)
#         self.conv4 = nn.conv2d(x, self.f)
    
        self.conv1_bn = nn.BatchNorm2d(128)
        self.conv2_bn = nn.BatchNorm2d(64)
        self.conv3_bn = nn.BatchNorm2d(48)

    def forward(self, X, verbose = False):
        X = self.conv1(X)
        X = F.selu(X)
        X = self.conv1_bn(X)
        
        X = self.conv2(X)
        X = F.selu(X)
        X = self.conv2_bn(X)

        
        X = self.conv3(X)
        X = F.selu(X)
        X = self.conv3_bn(X)
        
        X = self.conv4(X)
        
        return X

In [7]:
training_loss_list = []

def get_n_params(model):
    n = 0
    for p in list(model.parameters()):
        n += p.nelement()
    return n

def print_progress(epoch, batch_idx, data, train_loader, loss):
    training_loss_list.append(loss.item())
    if batch_idx % 1 == 0:
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tMSE Loss: {:.6f}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.item()))

In [8]:
def train(epoch, model):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.float().to(device), target.float().to(device)
        
        data = data.view(-1, 1, 160, 160)
        target = target.view(-1, 1, 160, 160)
        
        #BACKPROP
        output = model(data)
        loss = F.mse_loss(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        print_progress(epoch, batch_idx, data, train_loader, loss)
            
            
def test(model):
    model.eval()
    test_loss = 0
    n_samples = len(test_loader.dataset)
    
    for data, target in test_loader:
        data, target = data.float().to(device), target.float().to(device)
        test_loss += F.mse_loss(output, target).item() # sum up batch loss      

    test_loss /= n_samples

    print('\nTest set: Average loss: {:.4f}'.format(test_loss))


In [ ]:
n_epochs = 1000
input_size = 160 * 160
output_size = 160 * 160

print("Train Samples: ", len(train_loader.dataset))
print("Test Samples: ", len(test_loader.dataset), '\n')

model_cnn = CNN(input_size, output_size)
model_cnn.float().to(device)
optimizer = optim.Adam(model_cnn.parameters(), lr = 0.0001)

print('Number of parameters: {}'.format(get_n_params(model_cnn)))

for epoch in range(n_epochs):
    train(epoch, model_cnn)

test(model_cnn)

Train Samples:  3244
Test Samples:  406 

Number of parameters: 103440
Train Epoch: 0 [0/3244 (0%)]	MSE Loss: 1.385844
Train Epoch: 0 [64/3244 (2%)]	MSE Loss: 0.856997
Train Epoch: 0 [128/3244 (4%)]	MSE Loss: 0.709529
Train Epoch: 0 [192/3244 (6%)]	MSE Loss: 0.745747


In [ ]:
plt.plot(training_loss_list)